In [ ]:
import Thymio as th
import time
import costmap_path as cp
import cv2
import numpy as np

#FIXME: not right original image
original_image = cv2.imread('map.png',0)
costmap, block_size, start, goal, display_image=cp.init(original_image)
path,costmap = cp.update(costmap, start, goal, display_image)
next_goal = path[0]
current_pos = np.array([0,0,0])
current_speed = np.array([0,0,0])
camera_position = (0,0,0)
z_predicted = np.concatenate([current_pos,current_speed]) #kalman_state
sigma_predicted = np.diag([0.1,0.1,0.1,0,0,0])
thymio = th.Thymio()
camera_on = True
while True:
    
    #call camera to get image and update 
    
    #call function to get position of thymio from camera
    
    #call PID controller

    current_pos = z_predicted[:2] # Is used for the controller

    
    #call kallman filter

    z_filtered, sigma_filtered= thymio.filtering_step(z_k_k_1 = z_predicted, 
                                    sigma_k_k_1 = sigma_predicted, 
                                    V_left_measure = V_left_measured, 
                                    V_right_measure = V_right_measured, 
                                    camera_working=camera_on, 
                                    x_measured=x_observed, 
                                    y_measured=y_observed,
                                    theta_measured= theta_observed)
    


    #call kalmann prediction to update the sate and get current position in (x,y,theta)

    z_predicted, sigma_predicted = thymio.prediction_step(z_filtered, sigma_filtered)

   
    obstacle,detected = th.getObstaclePosition()
    if detected:
        print("Obstacle detected")
        #TODO: use the obstacle position to update the costmap
        path,costmap = cp.update(costmap, current_pos, goal, display_image)
        
        #FIXME:need to set threshold for proximity to obstacle
    if current_pos == next_goal:
        path.pop(0)
        if len(path) == 0:
            print("Goal reached")
            break
        next_goal = path[0]
        
    
    
    
    
    
    
    
    
 







NameError: name 'main' is not defined